# Lab \#1 Exploring Exploratory Data Analysis

Based on https://leanpub.com/raes, by Patrick Applegate (Copyright 2016). Adapted for Python and extended by Tony Wong (2022).

## Learning objectives

After completing this exercise, you should be able to 
* explain the relationship between atmospheric carbon dioxide concentration, surface air temperature, and sea level
* read data from a remote repository into Python/Pandas
* make simple and more complex plots in Python, including
  * plots with multiple panels
  * plots with multiple curves/sets of scatter points in the same panel
  * histograms
  * boxplots
  * time series/scatter plots
* compare plots and draw conclusions based on these comparisons

## Introduction

Throughout this course, we will discuss sea level rise extensively. Many people live near present-day sea level (e.g., Nicholls et al., 2008), and rises in sea level expose these people to the possibility of flooding. For example, the ongoing increase in sea level will likely cause communities on the United States’ eastern coast to experience frequent flooding within the next few decades ([Spanger-Siegfried et al., 2014](https://www.ucsusa.org/resources/encroaching-tides#.V1rbxb4rJBw)).

Sea level rise is caused by temperature increases, which in turn are driven by increases in carbon dioxide concentrations in the atmosphere. Carbon dioxide is produced by human activities and natural processes. Increases in carbon dioxide concentrations in the atmosphere enhance the trapping of infrared radiation near the Earth’s surface and contribute to rises in surface air temperatures. As the ocean absorbs some of the excess heat from the atmosphere, its temperature increases, causing it to expand and causing sea level rise. Temperature increases cause melt of glaciers and ice sheets, which leads to sea level rise by adding mass to the oceans.

Data covering the last century support this relationship between atmospheric carbon dioxide concentrations, temperature, and sea level (Fig. 1). The curves in the three panels of Figure 1 rise together, suggesting that these variables are related. Although correlation does not prove causation, the combination of a clear relationship between variables with a plausible explanation for why they should be related is a strong argument for causation.

> <div>
> <img src="https://github.com/tonyewong/risk_analysis_course_spring2022/blob/main/fig3.png?raw=true" width="500"/>
> </div>
>
> **Figure 1:** Atmospheric carbon dioxide concentrations (top panel), surface air temperature change (middle panel), and sea level change (bottom panel), between 1900 and ~2015. All three quantities rise over this period, suggesting a causal relationship between them. See text for discussion.

Frequent, accurate measurements of carbon dioxide in the atmosphere began in the late 1950s at Mauna Loa in Hawaii (Keeling et al., 1976; blue curve in Fig. 1, top panel). These measurements show an annual cycle that represents Northern Hemisphere seasons. Plants lose their leaves or die during the winter, releasing carbon dioxide to the atmosphere. The Northern Hemisphere has much more land, and therefore more plants, than the Southern Hemisphere. Thus, the Northern Hemisphere’s seasons largely control the variability in atmospheric carbon dioxide concentrations within any individual year. However, there is a definite upward trend in this curve that is larger than the amplitude of the annual cycle.

Measurements of former atmospheric compositions from bubbles trapped in ice cores allow us to extend the observational record of carbon dioxide concentrations farther back in time (MacFarling Meure et al., 2006; black curve in Fig. 1, top panel). As snow falls on the Greenland and Antarctic ice sheets, it traps samples of the atmosphere. Because new snow buries and compresses old snow, the time at which different snow samples fell can be estimated by counting the layers in an ice sheet. The ice core measurements of atmospheric carbon dioxide concentrations are less finely resolved in time than the direct measurements, and therefore don’t reflect the annual cycle of CO2 in the atmosphere. However, the trend of the ice core data is similar to that of the direct observations during the period when they overlap, suggesting that the ice core data are reliable.

Because carbon dioxide mixes readily in the atmosphere, measurements of atmospheric carbon dioxide concentrations at most places on the Earth’s surface are relatively representative of the globe as a whole. In contrast, both surface air temperatures and sea levels are measured at widely dispersed stations and must be aggregated to give a global mean value. Global mean temperatures must be estimated from individual weather stations with long records (Hansen et al., 2010); past sea levels are estimated using data from tide gages (Jevrejeva et al., 2014). As one might expect, there are various methods for performing this aggregation, and the different methods give somewhat different answers. However, it seems clear that both global mean surface air temperature and global mean sea level are rising.

In this lab exercise, you'll work with the data files needed to make the top two panels of Figure 1. You'll then modify your code so that it plots all three panels of Figure 1.

## Tutorial

We'll start by looking at several Web pages that describe the data we’ll be using in this exercise. As of May 2015, the various data sets displayed in Figure 1 are archived in the following places:

> **Table 1:** Internet sources of data used in Exercise \#1 and associated references. NOAA, National Oceanic and Atmospheric Administration Global Monitoring Laboratory; DOE CDIAC, NOAA National Centers for Environmental Information; NASA GISS, National Aeronautics and Space Administration – Goddard Institute for Space Studies; PSMSL, Permanent Service for Mean Sea Level.
>
> | Data type | Reference | Location on the Web |
|-----------|-----------|---------------------|
| CO$_2$ direct measurements | Keeling et al. (1976) | [NOAA GML](https://gml.noaa.gov/ccgg/trends/) |
| CO$_2$ Law Dome ice core | MacFarling Meure et al. (2006) | [NOAA NCEI](https://www.ncei.noaa.gov/access/paleo-search/study/9959) |
| Surface air temperature change | Hansen et al. (2010) | [NASA GISS](https://data.giss.nasa.gov/gistemp/)|
| Sea level anomaly | Jevrejeva et al. (2014) | [PSMSL](https://www.psmsl.org/products/reconstructions/jevrejevaetal2014.php) |

Click on the links in the right-hand column of Table 1 and look at the descriptions of the data stored there. Also look for links to the particular data sets mentioned in the Introduction. Some Web pages contain links to multiple data sets; we want the "Mauna Loa CO2 monthly mean data," the Law Dome data (scroll down to near the bottom of the page), and the “global-mean monthly, seasonal, and annual means, 1880-present, updated through most recent month” of the "Land-Ocean Temperature Index, LOTI."

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Reading in the GISS temperature data:** We can use a link from the NASA GISS site above, and the `read_csv` function from Pandas. For many data sets, we will need to skip over *metadata* rows at the top of a file. These are rows dedicated to giving the data users a sense of where the data came from, how the data set was processed, if there are any interpolated values or missing values, and other things of that nature. The `skiprows` argument tells Python how many rows to skip over. If you check out the actual CSV file by opening it in Google Sheets, Excel, or any text editor, you'll see why we need to skip one row here.

In [ ]:
dfT = pd.read_csv("https://data.giss.nasa.gov/gistemp/tabledata_v4/GLB.Ts+dSST.csv", 
                  skiprows=1)
dfT.head()

**Reading in the Law Dome ice core CO2 data:** We also use the `delimiter` argument to let Python know to expect spaces (`\s`) to separate actual table elements, but it might be any number of spaces (so we use `\s+`). The `encoding` argument there helps make the function more robust against strange/unexpected characters (e.g., a negative sign that's an em-dash instead of a regular hyphen, or exotic fonts).

In [ ]:
dfC_law = pd.read_csv("https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/law/law2006.txt",
                      skiprows=182, nrows=2004, delimiter="\s+", encoding = 'unicode_escape')
dfC_law.head()

### Task 1:

Check out the raw `law2006.txt` data file. Why did we use the `nrows=2004` argument to the `read_csv` command?

<br>

**Reading in the Mauna Loa CO2 data:** Note that you can also specify what you want/expect the column names to be by using the `names` argument.

In [ ]:
dfC_loa = pd.read_csv("https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv", 
                      skiprows=52, names=["year","month","decimal date","average","interpolated","trend","ndays",""])
dfC_loa.head()

<br>

### Task 2:

In the next task, you will need to re-create the middle panel of Fig. 1. Which column from the temperature DataFrame corresponds to the proper global annual mean temperature anomalies? How do you know?

<br>

### Task 3:

Finally, we can make a figure that reproduces the top two panels of Fig. 1. Below is code that reproduces the top panel of Fig. 1. Some things to note:
* The `subplots` command creates an array of plot `axis` objects that we can add plot elements to (like axis labels, curves, scatter points, etc.). Those `axis` objects are stored in the variable `ax`. We add to them using the `ax[#].plot` or `ax[#].set...` member function notation.
* If you want to manipulate the second panel, you can use `ax[1]` (since Python indexing starts at 0), and all these same functions to manipulate the plot.
* You will need to consider what columns of the temperature DataFrame `dfT` you want to use to plot. The first argument to `plot()` is the x elements, and the second argument gives the y elements.
* If you aren't sure what certain things do, **play around with changing them!** Or check the documentation by using your favorite search engine.

Using the column you decided on in Task 2, re-create the middle panel of Fig. 1, showing the global annual mean temperature anomalies over the last ~century. Be sure to label your axes, including units, where appropriate. Orient your two panels vertically (one on top of the other), and be sure that the x-axis limits match for all panels so that it's easy to compare what was going on at different points in time. You should modify the code cell below to include your solution code. Use a color for the temperature data that is not black, because that is boring, and we are not boring.

In [ ]:
fig,ax = plt.subplots(nrows=2, ncols=1, figsize=(8,9))

ax[0].plot(dfC_law["YearAD"], dfC_law["CO2spl"], color="black", label="Law Dome ice core record")
ax[0].plot(dfC_loa["decimal date"], dfC_loa["average"], color="steelblue", label="Mauna Loa measurements")
ax[0].set_xlim([1900,2020])
ax[0].set_xlabel("Time (yr)")
ax[0].set_ylabel("Atmospheric CO2 (ppm)")
ax[0].legend();

# YOUR CODE GOES HERE!

<br>

### Task 4:

The NASA GISS temperature data set that we are using is a few years longer than when this exercise was originally created ~2016 (because we are in the future). Compare your figure from Task 3 to Fig. 1. Is there anything noteworthy about the new temperature data points? (That is, the temperature data that are in your figure, but not in Fig. 1.)

<br>

### Task 5:

Read in the global sea level data from the fourth row of Table 1 using a combination of i) pattern-matching from the code above for reading in the data files, ii) looking at the raw global sea level data linked from Table 1, and iii) your sweet Python/Pandas skills. To be safe, the first and last lines of the data file should look like:
```
first line: 1807.5417	-5.597500	2.371786	-5.597500	55.195997
late line:  2010.4583	0.496430	0.790595	210.441283	14.710927
```
How you address the columns/column names is up to you! To be consistent with our previous DataFrame naming convention, call the DataFrame `dfS` and use `print(dfS)` to print the first few and last few rows to the screen. Be sure to leave your code cells executed before submitting your work, so that the output is visible without needing to re-run your notebook.

<br>

### Task 6:

Copy-paste your solution code from Task 3 below, and add to it so that you fully recreate Fig. 1 in all of its glory. Again, be sure to label your axis, including units where appropriate. You will need to make some modifications in order to add a third panel, and you might want to change the `figsize` argument in the `subplots()` function to accommodate this new panel. Again, orient your two panels vertically (one on top of another), and be sure that the x-axis limits match for all panels so that it's easy to compare what was going on at different points in time. Note that the `gsl`, not the `gsl_rate`, is what we want to plot from the sea level data set. 

<br>

### Task 7:

By how much have atmospheric carbon dioxide concentrations, global mean temperatures, and sea levels changed between 1900 and the early part of the present century (2000-present)? First, estimate this by using your figure from Task 6.

<br>

### Task 8:

Next, estimate the changes from Task 7 by comparing the means of each for the 11-year periods centered at 1900 and at 2000. That is, for each of the three data sets, compare the means for 1895-1905 and 1995-2005. This method of comparing centered averages is more robust than the method of Task 7, where natural variability from year to year in these times series conspires to make comparing individual years complicated. Write a **complete** sentence or two to interpret and summarize your findings.

_Hint: Use the `df.loc[...]` method to extract only the rows that meet specific criteria. The `NumpyPandasTutorial.ipynb` might be helpful there! As an example, if we wanted to grab only the Law Dome ice core data from 1990 and beyond, to get just those CO2 measurements we might say:_
```
dfC_law.loc[(dfC_law["YearAD"] >= 1990), "CO2spl"]
```

<br>

### Task 9:

It's a bit tough to see whether that is a big change in CO2. It's less than 100 ppm! That doesn't seem so bad, does it?

First, make a plot of the entier CO2 time series from the Law Dome ice core.

Then, compare the change over the last century (Task 8) to the difference between the maximum and minimum CO2 concentrations from the year 0 to the year 1900. _Yes, that is the actual year 0. The Law Dome ice core is over a km long, drilled down almost to the Antarctic continent's bedrock under all that ice, and goes back tens of thousands of years!_ Write a **complete** sentence or two to interpret and summarize your findings.

<br>

---

## Changing gears a little bit...

Consider the _extreme sea level_ data set `KeyWest-AnnualMaximumSeaLevels.csv`, which may be found on my GitHub at the URL in the code snippet below. These are the actual annual maximum sea levels (in excess of a particular threshold that isn't important here) for the tide gauge station in Key West, Florida, from 1913 to 2019. The first column of the data set is the `year` and the second column is the annual maximum sea level `height` (in millimeters). I have detrended the original data set to remove the effect of mean sea-level rise. You also do not need to worry about waves and tides. 

In [ ]:
dfK = pd.read_csv("https://raw.githubusercontent.com/tonyewong/risk_analysis_course_spring2022/main/KeyWest-AnnualMaximumSeaLevels.csv")
year = dfK["year"]
esl = dfK["height"]


<br>

### Task 10

What is the median and interquartile range? (You can round your values to the nearest millimeter.) Based on these three values, how would you characterize the skew of the distribution, and why? Write a couple of complete sentences to explain your reasoning.

<br>

### Task 11

Make a density histogram of these extreme sea levels. Label all axes, including units, where appropriate. How would you characterize this distribution in terms of number of modes and skew?

<br>

### Task 12

As the IPCC chapter "Managing The Risks Of Extreme Events And Disasters To Advance Climate Change Adaptation Summary For Policymakers" (reading for today!) noted, extreme coastal sea levels are a potentially increasing hazard in a changing climate. Use side-by-side boxplots in a single figure panel and the data set from this problem (`year` and `esl`) to demonstrate that extreme sea levels have gotten more severe over time. Make sure to clearly label all plot elements in your figure, including units where applicable. Write a few sentences to describe and interpret your results.

<br>

### Task 13

Use some of our exploratory data analysis methods from this course and the data set from this problem (`year` and `esl`) to demonstrate that extreme sea levels have ***not*** gotten more severe over time. Write a few sentences to describe what you are doing, describing the results, and interpreting the results. Make sure to clearly label all plot elements in any figures that you generate, including units where applicable.